# ETL — Accidents de la route

Ce notebook montre, pas à pas, comment :
1) se connecter à l’API publique Opendatasoft,
2) récupérer un petit échantillon,
3) paginer pour extraire un volume plus grand,
4) sauvegarder les données brutes en CSV,
5) poser les bases du nettoyage (à faire en équipe).

> **Pourquoi ce format ?**  
> Un notebook est idéal pour apprendre : on alterne **explications** (Markdown) et **code** (Python), et on voit les résultats immédiatement.

In [ ]:
# Imports
import pandas as pd
import requests
import time
from pathlib import Path

# Dossiers de sortie
RAW_DIR = Path("../data/raw")
CLEAN_DIR = Path("../data/cleaned")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)


BASE_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

SELECT_PARTS = [
    "num_acc",
    "datetime",
    "an",
    "mois",
    "jour",
    "hrmn",
    "lum",
    "agg",
    '"int" as intersection',   # <= réservé → cité + alias
    "atm",
    "col",
    "dep",
    "com",
    "insee",
    "adr",
    "lat",
    '"long" as lon',           # <= réservé → cité + alias
    "surf",
    "circ",
    "nbv",
    "catr",
    "plan",
    "prof",
    "infra",
    "situ",
    "gps",
    "year_georef",
    "dep_name",
    "reg_name",
    "epci_name"
]
SELECT = ", ".join(SELECT_PARTS)


In [ ]:
def fetch_page(offset=0, limit=1000):
    params = {
        "select": SELECT,
        "limit": limit,
        "offset": offset,
        "order_by": "datetime"   # champ 'safe' pour trier
    }
    r = requests.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    return r.json().get("results", [])

# Test rapide
sample = fetch_page(0, 10)
df = pd.DataFrame(sample)
df.head(3)

In [ ]:
TARGET = 1000
PAGE = 100
DELAY = 0.3

all_rows = []
offset = 0
while offset < TARGET:
    chunk = fetch_page(offset, PAGE)
    if not chunk:
        break
    all_rows.extend(chunk)
    offset += PAGE
    time.sleep(DELAY)

# Sauvegarder un export "brut" pour traçabilité
df_raw = pd.DataFrame(all_rows)
df_raw.to_csv(RAW_DIR / "accidents_sample_raw.csv", index=False)
print(f"{len(df_raw)} lignes enregistrées")


### Chargement des données sources

Jusqu’ici, nous avons vu comment interagir avec l’API publique d’Opendatasoft pour récupérer les données dont nous avons besoin.  
Un fichier d’exemple de 1 000 lignes a permis d’illustrer le principe de pagination et de test de l’API.

Cependant, l’API limite les extractions à des paquets de 100 enregistrements, et le jeu complet (plus de 500 000 lignes) aurait demandé un temps de traitement trop important.  

Pour la suite du brief, nous utiliserons donc directement le fichier CSV complet, déjà téléchargé et préparé à partir de la source officielle :  
👉 [Accidents corporels de la circulation millésimé — Opendatasoft](https://public.opendatasoft.com/explore/assets/accidents-corporels-de-la-circulation-millesime/export/)

Ce fichier servira de base à toutes les étapes suivantes de notre pipeline (nettoyage, transformation et analyse).

# # Nettoyage du jeu de données brut
# Ce bloc charge le fichier CSV brut téléchargé depuis Opendatasoft,
# garde uniquement les colonnes utiles à notre modèle, puis écrit un CSV nettoyé dans `data/raw`.

In [ ]:
import pandas as pd

# 1) Lire (essaie UTF-8 puis fallback latin1 si besoin)
try:
    df = pd.read_csv("../data/raw/accidents-corporels-de-la-circulation-millesime.csv",
                     sep=";", dtype=str, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv("../data/raw/accidents-corporels-de-la-circulation-millesime.csv",
                     sep=";", dtype=str, encoding="latin1")

print("Colonnes détectées:", list(df.columns)[:8], "...")

# 2) Garder seulement les colonnes utiles (sans renommage)
cols = [
    "Identifiant de l'accident","Date et heure","Commune","Année","Mois","Jour","Heure minute",
    "Lumière","Localisation","Intersection","Conditions atmosphériques","Collision",
    "Département","Code commune","Code Insee","Adresse","Latitude","Longitude",
    "Surface","Circulation","Nombre de voies","Catégorie route","Plan","Profil",
    "Infrastructure","Situation","Gps","year_georef",
    "Nom Officiel Département","Nom Officiel Région","Nom Officiel EPCI","Nom Officiel Commune",
]
df_filtered = df[cols]

# 3) Écrire pour Excel → UTF-8 avec BOM + pas d’index
out_path = "../data/raw/accidents-corporels-de-la-circulation-millesime_raw.csv"
df_filtered.to_csv(out_path, sep=";", index=False, encoding="utf-8-sig")

print("✅ CSV créé (compatible Excel) :", out_path)


### Nettoyage de base

Nous allons maintenant procéder à un premier nettoyage des données.  
L’objectif ici est d’obtenir un fichier exploitable et cohérent avant d’entamer les transformations plus avancées.

Ce nettoyage de base consiste à :
- supprimer les doublons éventuels,
- éliminer les lignes sans identifiant d’accident,
- préparer un fichier CSV propre dans le dossier `data/cleaned`.

Cette étape garantit que les traitements suivants (analyse, enrichissement, agrégations) reposeront sur un jeu de données unique et fiable.


A PAUFINER

In [ ]:

import pandas as pd
from pathlib import Path

# Charger le fichier déjà filtré
df_raw = pd.read_csv(
    "../data/raw/accidents-corporels-de-la-circulation-millesime_raw.csv",
    sep=";",
    dtype=str,
    encoding="utf-8-sig"
)

print("✅ Fichier chargé :", len(df_raw), "lignes")
print("Colonnes :", list(df_raw.columns)[:8], "...")

# Règles minimales : garder une clé, dédoublonner
df_clean = df_raw.copy()

# supprimer les lignes sans identifiant d'accident
# le nom exact de la colonne est "Identifiant de l'accident"
if "Identifiant de l'accident" in df_clean.columns:
    df_clean = (
        df_clean.dropna(subset=["Identifiant de l'accident"])
                .drop_duplicates(subset=["Identifiant de l'accident"])
    )

print("✅ Après nettoyage :", len(df_clean), "lignes")

# Sauvegarde du jeu nettoyé
CLEAN_DIR = Path("../data/cleaned")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

clean_csv = CLEAN_DIR / "accidents_clean.csv"
df_clean.to_csv(clean_csv, sep=";", index=False, encoding="utf-8-sig")

print("💾 Fichier nettoyé enregistré :", clean_csv)


## Prochaines étapes 

- Générer `dim_time` à partir des dates (ou d’un calendrier)  
- Créer `dim_location` (distinct de `codeinsee`, `departement`, lat/lon si dispo)  
- Créer `dim_conditions` (distinct de `lumiere`, `meteo`, `type_route`, `type_de_collision`)  
- Mapper les IDs (clé naturelle → clé de substitution)  
- Remplir `fact_accident` avec les mesures (`ttue`, `tbg`, `tbl`, `tindm`, `grav`)  
- Écrire `sql/schema.sql` et (option) charger via SQLAlchemy dans SQLite/MySQL/PostgreSQL
